In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from tqdm import tqdm
import random
import json 

In [2]:
#Reading the dataset
data = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/output/Movies_dataset.csv')
data.head()

,Unnamed: 0,movieId,imdbId,tmdbId,title,overview,budget,genres,production_companies,production_countries,release_date,runtime,spoken_languages,keywords,cast,crew,final_poster_url
0,0,1,114709,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",NF
1,1,2,113497,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",https://image.tmdb.org/t/p/w500/vzmL6fP7aPKNKP...
2,2,3,113228,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",NF
3,3,4,114885,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",NF
4,4,5,113041,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...,0.0,"[{'id': 35, 'name': 'Comedy'}]","[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",NF


In [4]:
def extract_genre(x):
    x_list = eval(x)
    genre = []
    for i in x_list:
        genre.append(i['name'])
    return genre


data['genre_list'] = data['genres'].apply(extract_genre)
data.head()

,Unnamed: 0,movieId,imdbId,tmdbId,title,overview,budget,genres,production_companies,production_countries,release_date,runtime,spoken_languages,keywords,cast,crew,final_poster_url,genre_list
0,0,1,114709,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",NF,"[Animation, Comedy, Family]"
1,1,2,113497,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",https://image.tmdb.org/t/p/w500/vzmL6fP7aPKNKP...,"[Adventure, Fantasy, Family]"
2,2,3,113228,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",NF,"[Romance, Comedy]"
3,3,4,114885,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",NF,"[Comedy, Drama, Romance]"
4,4,5,113041,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...,0.0,"[{'id': 35, 'name': 'Comedy'}]","[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",NF,[Comedy]


In [5]:
def check_genre(x):
    if 'Animation' in x:
        return 'Animation'
    elif 'Comedy' in x:
        return 'Comedy'
    elif 'Action' in x:
        return 'Action'
    

data['genre_filter'] = data['genre_list'].apply(check_genre)
data.head()

,Unnamed: 0,movieId,imdbId,tmdbId,title,overview,budget,genres,production_companies,production_countries,release_date,runtime,spoken_languages,keywords,cast,crew,final_poster_url,genre_list,genre_filter
0,0,1,114709,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",NF,"[Animation, Comedy, Family]",Animation
1,1,2,113497,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",https://image.tmdb.org/t/p/w500/vzmL6fP7aPKNKP...,"[Adventure, Fantasy, Family]",None
2,2,3,113228,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",NF,"[Romance, Comedy]",Comedy
3,3,4,114885,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",NF,"[Comedy, Drama, Romance]",Comedy
4,4,5,113041,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...,0.0,"[{'id': 35, 'name': 'Comedy'}]","[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",NF,[Comedy],Comedy


### Filtering out MovieIds for Animation

In [6]:
Animation_movies = data[data['genre_filter'] == 'Animation']['movieId'].to_list()
len(Animation_movies)

1917

### Filtering out MovieIds for Comedy

In [7]:
Comedy_movies = data[data['genre_filter'] == 'Comedy']['movieId'].to_list()
len(Comedy_movies)

12253

### Filtering out MovieIds for Action

In [8]:
Action_movies = data[data['genre_filter'] == 'Action']['movieId'].to_list()
len(Action_movies)

4957

In [9]:
#Read rating files
ratings = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data_big/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [6]:
#Filter the ratings for those movies present in 
filter_ratings = ratings[ratings['movieId'].isin(data['movieId'])]
filter_ratings

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [7]:
filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)

/var/folders/qt/lkmkr1817vd45bbs2t4fbrv00000gn/T/ipykernel_7033/725361958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)


In [8]:
filter_ratings.to_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_all.dat', sep=':', index=False, header=False)

Above is the userId for some 1000 users and corresponding movieId - For now computing database of only 1000 users.

In [9]:
data_df = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_all.dat', sep=':', names=["UserID", "MovieID", "Rating"], engine='python')

In [10]:
data_df

,UserID,MovieID,Rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
25976892,270896,58559,5.0
25976893,270896,60069,5.0
25976894,270896,63082,4.5
25976895,270896,64957,4.5


In [11]:
# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['MovieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
np.random.seed(12)
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

In [12]:
data_df

,UserID,MovieID,Rating
0,0,0,1.0
1,0,1,4.5
2,0,2,5.0
3,0,3,5.0
4,0,4,5.0
...,...,...,...
25976892,270881,14,5.0
25976893,270881,519,5.0
25976894,270881,168,4.5
25976895,270881,532,4.5


In [13]:
train_mat.shape

(270882, 43729)

In [14]:
test_mat.shape

(270882, 43729)

In [15]:
train_mat

array([[1., 0., 5., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 4., ..., 0., 0., 0.],
       ...,
       [0., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 5., ..., 0., 0., 0.]])

In [11]:
class MF_withBias:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  
        self.test_mat = test_mat  

        self.latent = latent  
        self.lr = lr 
        self.reg = reg 

        self.num_user, self.num_movie = train_mat.shape

        self.sample_user, self.sample_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat
        self.num_sample = len(self.sample_user)  # the number of user-movie pairs having ratings in train_mat

        self.train_indicator_mat = 1.0 * (train_mat > 0)  # binary matrix to indicate whether s user-movie pair has rating or not in train_mat
        self.test_indicator_mat = 1.0 * (test_mat > 0)  # binary matrix to indicate whether s user-movie pair has rating or not in test_mat

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        self.user_bias = np.random.random(self.num_user)
        self.movie_bias = np.random.random(self.num_movie)

    def train(self, epoch=20, verbose=True):
       
        epoch_loss_list = []
        epoch_test_RMSE_list = []
        indices = list(range(self.num_sample))
        for ep in tqdm(range(epoch)):
         
            random.shuffle(indices)
            total_loss = 0.0
            for index in indices:
                u = self.sample_user[index]
                i = self.sample_movie[index]

                rating = self.train_mat[u,i]
                prediction = np.dot(self.P[u], self.Q[i].T) + self.user_bias[u] + self.movie_bias[i]

                error = prediction - rating
                user_learn = 2 * error * self.Q[i] + 2 * self.reg * self.P[u]
                movie_learn = 2 * error * self.P[u] + 2 * self.reg * self.Q[i]
                user_bias_learn = 2 * error + 2 * self.reg * self.user_bias[u]   
                movie_bias_learn = 2 * error + 2 * self.reg * self.movie_bias[i] #Adding Bias terms

                self.P[u] = self.P[u] - self.lr * user_learn
                self.Q[i] = self.Q[i] - self.lr * movie_learn
                self.user_bias[u] = self.user_bias[u] - self.lr * user_bias_learn
                self.movie_bias[i] = self.movie_bias[i] - self.lr *  movie_bias_learn

                total_loss += error**2

            
            prediction_mat = np.matmul(self.P, self.Q.T) + self.user_bias[:, np.newaxis] + self.movie_bias[np.newaxis, :]
            test_mat_nan = np.where(self.test_mat == 0, np.nan, self.test_mat)
            rmse = np.sqrt(np.nanmean((prediction_mat - test_mat_nan)**2))
            avg_train_loss = total_loss / self.num_sample
            epoch_loss_list.append(avg_train_loss)
            epoch_test_RMSE_list.append(rmse)

            if verbose:
                print(f"Epoch {ep+1}: Average Training Loss = {avg_train_loss} | Test RMSE = {rmse}")

        return epoch_loss_list, epoch_test_RMSE_list


    def predict(self):
        prediction_mat = np.matmul(self.P, self.Q.T) + self.user_bias[:, np.newaxis] + self.movie_bias[np.newaxis, :]
        return prediction_mat

In [17]:
mf = MF_withBias(train_mat, test_mat)
epoch_loss_list, epoch_test_RMSE_list = mf.train(epoch=20, verbose = False)

: 

In [ ]:
preds = mf.predict()

In [ ]:
type(preds)

numpy.ndarray

In [ ]:
movie_id = np.argsort(np.mean(preds, axis =0))

In [ ]:
arr = movie_id[::-1][:5]

In [ ]:
arr

array([1732, 1726,  411, 1513, 1980])

In [ ]:
# Function to find keys for given values in a dictionary
def find_keys(dictionary, values):
    keys = []
    for key, value in dictionary.items():
        if value in values:
            keys.append(key)
    return keys

# Find keys in each dictionary
keys_dict1 = find_keys(movie_old2new_id_dict, arr)
keys_dict1 = [int(key) for key in keys_dict1]



In [ ]:
import json 

path = '/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/output/Global_top5.json'

json_str = json.dumps(keys_dict1)
with open(path, 'w') as f:
    f.write(json_str)

### Animation movies

In [12]:
filter_ratings = ratings[ratings['movieId'].isin(Animation_movies)]
filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)
filter_ratings.to_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_Animation.dat', sep=':', index=False, header=False)
data_df = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_Animation.dat', sep=':', names=["UserID", "MovieID", "Rating"], engine='python')

/var/folders/qt/lkmkr1817vd45bbs2t4fbrv00000gn/T/ipykernel_5959/2844776031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)


In [13]:
#make the matrics
# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['MovieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
np.random.seed(12)
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

In [14]:
#MF 
mf = MF_withBias(train_mat, test_mat)
epoch_loss_list, epoch_test_RMSE_list = mf.train(epoch=20, verbose = False)
preds = mf.predict()

100%|██████████| 20/20 [05:26<00:00, 16.30s/it]


In [15]:
movie_id = np.argsort(np.mean(preds, axis =0))
arr = movie_id[::-1][:5]


# Function to find keys for given values in a dictionary
def find_keys(dictionary, values):
    keys = []
    for key, value in dictionary.items():
        if value in values:
            keys.append(key)
    return keys

# Find keys in each dictionary
keys_dict1 = find_keys(movie_old2new_id_dict, arr)
keys_dict1 = [int(key) for key in keys_dict1]



path = '/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/output/Animation_top5.json'

json_str = json.dumps(keys_dict1)
with open(path, 'w') as f:
    f.write(json_str)



### Comedy Movies

In [16]:
filter_ratings = ratings[ratings['movieId'].isin(Comedy_movies)]
filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)
filter_ratings.to_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_Comedy.dat', sep=':', index=False, header=False)
data_df = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_Comedy.dat', sep=':', names=["UserID", "MovieID", "Rating"], engine='python')

/var/folders/qt/lkmkr1817vd45bbs2t4fbrv00000gn/T/ipykernel_5959/22946532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)


In [17]:
filter_ratings

,userId,movieId,rating
5,1,1968,4.0
7,1,2918,5.0
11,1,5577,5.0
13,1,54503,3.5
27,2,5,3.0
...,...,...,...
26024273,270896,50954,2.5
26024274,270896,51255,4.5
26024276,270896,52973,3.5
26024280,270896,54503,4.0


In [18]:
#make the matrics
# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['MovieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
np.random.seed(12)
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()


In [ ]:
#MF 
mf = MF_withBias(train_mat, test_mat)
epoch_loss_list, epoch_test_RMSE_list = mf.train(epoch=20, verbose = False)
preds = mf.predict()

: 

In [15]:
movie_id = np.argsort(np.mean(preds, axis =0))
arr = movie_id[::-1][:5]


# Function to find keys for given values in a dictionary
def find_keys(dictionary, values):
    keys = []
    for key, value in dictionary.items():
        if value in values:
            keys.append(key)
    return keys

# Find keys in each dictionary
keys_dict1 = find_keys(movie_old2new_id_dict, arr)
keys_dict1 = [int(key) for key in keys_dict1]






path = '/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/output/Comedy_top5.json'

json_str = json.dumps(keys_dict1)
with open(path, 'w') as f:
    f.write(json_str)



/var/folders/qt/lkmkr1817vd45bbs2t4fbrv00000gn/T/ipykernel_5169/1157581042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)


: 

In [ ]:
filter_ratings = ratings[ratings['movieId'].isin(Action_movies)]
filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)
filter_ratings.to_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_Action.dat', sep=':', index=False, header=False)
data_df = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_Action.dat', sep=':', names=["UserID", "MovieID", "Rating"], engine='python')



#make the matrics
# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['MovieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
np.random.seed(12)
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()



#MF 
mf = MF_withBias(train_mat, test_mat)
epoch_loss_list, epoch_test_RMSE_list = mf.train(epoch=20, verbose = False)
preds = mf.predict()



movie_id = np.argsort(np.mean(preds, axis =0))
arr = movie_id[::-1][:5]


# Function to find keys for given values in a dictionary
def find_keys(dictionary, values):
    keys = []
    for key, value in dictionary.items():
        if value in values:
            keys.append(key)
    return keys

# Find keys in each dictionary
keys_dict1 = find_keys(movie_old2new_id_dict, arr)
keys_dict1 = [int(key) for key in keys_dict1]






path = '/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/output/Action_top5.json'

json_str = json.dumps(keys_dict1)
with open(path, 'w') as f:
    f.write(json_str)

